<a href="https://colab.research.google.com/github/mafaldasalomao/pavic_treinamento_ml/blob/main/PAVIC_ML_16_PT_autoencoders_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 16: Autoencoders e classificação

## Etapa 1: Importação das bibliotecas

In [ ]:
from torchvision import datasets, transforms
import torch
from torch import nn, optim
from sklearn.metrics import accuracy_score
torch.__version__

'1.4.0'

In [ ]:
torch.manual_seed(123)

## Etapa 2: Base de dados

In [ ]:
dataset_train = datasets.MNIST('MNIST-data', train = True, download = True,
                               transform = transforms.ToTensor())
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 256,
                                           shuffle = True)


Extracting MNIST-data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST-data/MNIST/raw



Extracting MNIST-data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST-data/MNIST/raw



Extracting MNIST-data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST-data/MNIST/raw



Extracting MNIST-data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST-data/MNIST/raw
Processing...
Done!


In [ ]:
dataset_val = datasets.MNIST('MNIST-data', train = False, download = True,
                             transform = transforms.ToTensor())
loader_val = torch.utils.data.DataLoader(dataset_val, batch_size = 256,
                                         shuffle = True)

## Etapa 3: Construção do autoencoder

In [ ]:
# 784 -> 32 -> 784
class autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(in_features = 784, out_features = 32)
        self.activation0 = nn.ReLU()
        self.dense1 = nn.Linear(32, 784)
        self.activation1 = nn.Sigmoid()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dense1(X)
        X = self.activation1(X)
        return X

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
model = autoencoder()
model.to(device)

autoencoder(
  (dense0): Linear(in_features=784, out_features=32, bias=True)
  (activation0): ReLU()
  (dense1): Linear(in_features=32, out_features=784, bias=True)
  (activation1): Sigmoid()
)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

## Etapa 4: Treinamento do modelo

In [ ]:
for epoch in range(20):
    running_loss_train = 0.
    for data in loader_train:
        model.train()
        inputs, _ = data
        inputs = inputs.to(device)

        inputs = inputs.view(-1, 28*28)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, inputs.view(*outputs.shape))
        loss.backward()

        optimizer.step()

        running_loss_train += loss.item()

    # Validação
    running_loss_val = 0.
    for data in loader_val:
        model.eval()
        inputs, _ = data
        inputs = inputs.to(device)
        inputs = inputs.view(-1, 28*28)
        outputs = model(inputs)
        loss = criterion(outputs, inputs.view(*outputs.shape))
        running_loss_val += loss.item()

    # Final da época
    print('ÉPOCA {:3d}: perda_train {:.5f} perda_val {:.5f}'.format(epoch + 1, running_loss_train/len(loader_train), running_loss_val/len(loader_val)))

ÉPOCA   1: perda_train 0.29703 perda_val 0.20237
ÉPOCA   2: perda_train 0.18216 perda_val 0.16382
ÉPOCA   3: perda_train 0.15509 perda_val 0.14599
ÉPOCA   4: perda_train 0.14117 perda_val 0.13517
ÉPOCA   5: perda_train 0.13216 perda_val 0.12711
ÉPOCA   6: perda_train 0.12553 perda_val 0.12166
ÉPOCA   7: perda_train 0.12053 perda_val 0.11690
ÉPOCA   8: perda_train 0.11689 perda_val 0.11407
ÉPOCA   9: perda_train 0.11434 perda_val 0.11195
ÉPOCA  10: perda_train 0.11261 perda_val 0.11056
ÉPOCA  11: perda_train 0.11141 perda_val 0.10932
ÉPOCA  12: perda_train 0.11064 perda_val 0.10881
ÉPOCA  13: perda_train 0.11008 perda_val 0.10857
ÉPOCA  14: perda_train 0.10975 perda_val 0.10802
ÉPOCA  15: perda_train 0.10948 perda_val 0.10766
ÉPOCA  16: perda_train 0.10927 perda_val 0.10746
ÉPOCA  17: perda_train 0.10904 perda_val 0.10659
ÉPOCA  18: perda_train 0.10740 perda_val 0.10526
ÉPOCA  19: perda_train 0.10691 perda_val 0.10534
ÉPOCA  20: perda_train 0.10671 perda_val 0.10498


## Etapa 5: Geração do encoder

In [ ]:
list(model.children())

[Linear(in_features=784, out_features=32, bias=True),
 ReLU(),
 Linear(in_features=32, out_features=784, bias=True),
 Sigmoid()]

In [ ]:
class encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = list(model.children())[0]
        self.activation0 = list(model.children())[1]

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        return X

In [ ]:
model1 = encoder()
model1.to(device)

encoder(
  (dense0): Linear(in_features=784, out_features=32, bias=True)
  (activation0): ReLU()
)

## Etapa 6: Codificação dos previsores

In [ ]:
previsores_treinamento = dataset_train.data.float() / 255
previsores_treinamento = previsores_treinamento.view(-1, 784)
previsores_treinamento = previsores_treinamento.to(device)

In [ ]:
previsores_treinamento.shape

torch.Size([60000, 784])

In [ ]:
with torch.no_grad():
  previsores_treinamento_codificados = model1(previsores_treinamento)
classe_treinamento = dataset_train.targets

In [ ]:
previsores_treinamento_codificados.shape

torch.Size([60000, 32])

In [ ]:
classe_treinamento

tensor([5, 0, 4,  ..., 5, 6, 8])

In [ ]:
dataset_train_cod = torch.utils.data.TensorDataset(previsores_treinamento_codificados, classe_treinamento)
loader_train_cod = torch.utils.data.DataLoader(dataset_train_cod, batch_size = 256, shuffle=True)

In [ ]:
previsores_val = dataset_val.data.float() / 255
previsores_val = previsores_val.view(-1, 784)
previsores_val = previsores_val.to(device)
with torch.no_grad():
  previsores_val_codificados = model1(previsores_val)
classe_val = dataset_val.targets

In [ ]:
previsores_val_codificados.shape

torch.Size([10000, 32])

In [ ]:
dataset_test_cod = torch.utils.data.TensorDataset(previsores_val_codificados, classe_val)
loader_val_cod = torch.utils.data.DataLoader(dataset_test_cod, batch_size = 256, shuffle=True)

## Etapa 7: Classificação sem redução de dimensionalidade

In [ ]:
c1 = nn.Sequential(nn.Linear(784, 397),
                   nn.ReLU(),
                   nn.Linear(397, 10),
                   nn.LogSoftmax())
c1.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(c1.parameters())

In [ ]:
for epoch in range(20):
    # Treinamento
    running_loss_train = 0.
    running_accuracy_train = 0.
    for data in loader_train:
        c1.train()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 28*28)
        optimizer.zero_grad()
        outputs = c1(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss_train += loss.item()
        running_accuracy_train += accuracy_score(labels.detach().cpu().numpy(),
                                                 top_class.detach().cpu().numpy())

    # Validação
    running_loss_val = 0.
    running_accuracy_val = 0.
    for data in loader_val:
        c1.eval()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 28*28)
        outputs = c1(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        running_loss_val += loss.item()
        running_accuracy_val += accuracy_score(labels.detach().cpu().numpy(),
                                               top_class.detach().cpu().numpy())

    # Final da época
    print('ÉPOCA {:3d}: perda_train {:.5f} precisão_train {:5f} perda_val {:.5f} precisão_val {:5f}'.format(epoch + 1,
            running_loss_train/len(loader_train),
            running_accuracy_train/len(loader_train),
            running_loss_val/len(loader_val),
            running_accuracy_val/len(loader_val)))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


ÉPOCA   1: perda_train 0.41315 precisão_train 0.892509 perda_val 0.21626 precisão_val 0.937305
ÉPOCA   2: perda_train 0.18429 precisão_train 0.946742 perda_val 0.15411 precisão_val 0.954395
ÉPOCA   3: perda_train 0.12669 precisão_train 0.963115 perda_val 0.11340 precisão_val 0.965430
ÉPOCA   4: perda_train 0.09371 precisão_train 0.973055 perda_val 0.09151 precisão_val 0.972852
ÉPOCA   5: perda_train 0.07281 precisão_train 0.979239 perda_val 0.09010 precisão_val 0.972656
ÉPOCA   6: perda_train 0.05920 precisão_train 0.982923 perda_val 0.08068 precisão_val 0.975000
ÉPOCA   7: perda_train 0.04797 precisão_train 0.986569 perda_val 0.07780 precisão_val 0.974609
ÉPOCA   8: perda_train 0.04003 precisão_train 0.988431 perda_val 0.07258 precisão_val 0.976270
ÉPOCA   9: perda_train 0.03217 precisão_train 0.991384 perda_val 0.06726 precisão_val 0.977637
ÉPOCA  10: perda_train 0.02703 precisão_train 0.992631 perda_val 0.06395 precisão_val 0.979590
ÉPOCA  11: perda_train 0.02234 precisão_train 0.99

## Etapa 8: Classificação com redução de dimensionalidade

In [ ]:
c2 = nn.Sequential(
        nn.Linear(32, 21),
        nn.ReLU(),
        nn.Linear(21, 10),
        nn.LogSoftmax())
c2.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(c2.parameters())

In [ ]:
for epoch in range(20):
    running_loss_train = 0.
    running_accuracy_train = 0.
    for data in loader_train_cod:
        c2.train()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = c2(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss_train += loss.item()
        running_accuracy_train += accuracy_score(labels.detach().cpu().numpy(),
                                                 top_class.detach().cpu().numpy())

    # Validação
    running_loss_val = 0.
    running_accuracy_val = 0.
    for data in loader_val_cod:
        c2.eval()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = c2(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        running_loss_val += loss.item()
        running_accuracy_val += accuracy_score(labels.detach().cpu().numpy(),
                                               top_class.detach().cpu().numpy())

    # Final da época
    print('ÉPOCA {:3d}: perda_train {:.5f} precisão_train {:5f} perda_val {:.5f} precisão_val {:5f}'.format(epoch + 1,
            running_loss_train/len(loader_train),
            running_accuracy_train/len(loader_train),
            running_loss_val/len(loader_val),
            running_accuracy_val/len(loader_val)))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


ÉPOCA   1: perda_train 1.47602 precisão_train 0.533405 perda_val 0.70256 precisão_val 0.801562
ÉPOCA   2: perda_train 0.56294 precisão_train 0.842260 perda_val 0.47147 precisão_val 0.863867
ÉPOCA   3: perda_train 0.44143 precisão_train 0.869376 perda_val 0.39152 precisão_val 0.883496
ÉPOCA   4: perda_train 0.40128 precisão_train 0.879737 perda_val 0.36594 precisão_val 0.892871
ÉPOCA   5: perda_train 0.37880 precisão_train 0.886769 perda_val 0.34917 precisão_val 0.900098
ÉPOCA   6: perda_train 0.35935 precisão_train 0.892559 perda_val 0.34044 precisão_val 0.900000
ÉPOCA   7: perda_train 0.34318 precisão_train 0.896410 perda_val 0.31632 precisão_val 0.907324
ÉPOCA   8: perda_train 0.32880 precisão_train 0.901463 perda_val 0.30438 precisão_val 0.912305
ÉPOCA   9: perda_train 0.31766 precisão_train 0.904028 perda_val 0.31044 precisão_val 0.909668
ÉPOCA  10: perda_train 0.30737 precisão_train 0.908018 perda_val 0.28630 precisão_val 0.919727
ÉPOCA  11: perda_train 0.29784 precisão_train 0.91